In [ ]:
import pandas as pd 
import re
import geonamescache
from unidecode import unidecode

In [ ]:
f = open('./data/headlines.txt',"r")
headlines = f.read().splitlines()
f.close()

In [ ]:
for headline in headlines:
    print(headline)

In [ ]:
gc = geonamescache.GeonamesCache()
citiesNorm = []
citiesOrg = []
cityUnidecode = ""

for city in gc.get_cities().values():
    name = city['name']
    cityUnidecode = unidecode(city['name']).replace("'", "")
    citiesNorm.append(cityUnidecode)
    citiesOrg.append(city['name'])
        
citiesNorm.sort(key=lambda item: (-len(item), item), reverse=False)
citiesOrg.sort(key=lambda item: (-len(item), item), reverse=False)



_cities = "|".join(citiesNorm)
regexCity = "\\b(" + _cities + ")\\b"
compiledReCity = re.compile(regexCity)

In [ ]:
countriesNorm = []
countriesOrg = []
countryUnidecode = ""

for country in gc.get_countries().values():
    name = country['name']
    countryUnidecode = unidecode(country['name'])
    countriesNorm.append(countryUnidecode)
    countriesOrg.append(country['name'])
        
countriesNorm.sort(key=lambda item: (-len(item), item), reverse=False)
countriesOrg.sort(key=lambda item: (-len(item), item), reverse=False)



_countries = "|".join(countriesNorm)
regexCountry = "\\b(" + _countries + ")\\b"
compiledReCountry = re.compile(regexCountry)

In [ ]:
df = pd.DataFrame(columns =['headline', 'city', 'country']) 

In [ ]:
for headline in headlines:
    m = compiledReCity.search(headline)
    if m == None:
        n = compiledReCountry.search(headline)
        if n == None:
            df = df.append({'headline' : headline, "city" : "" , "country" : ""} , ignore_index=True)  
        else:
            _country = n.group(0)
            df = df.append({'headline' : headline, "city" : "" , "country" : _country} , ignore_index=True)    
    else:  
        _city = m.group(0)
        citiesMatch = {}
        for city in citiesOrg: 
            if _city == unidecode(city).replace("'", ""):
                _cities = gc.get_cities_by_name(city)
                for _c in _cities:
                    for key, value in _c.items(): 
                        citiesMatch[key] = value
        biggestCity = {}
        for cityKey in citiesMatch:
            if len(biggestCity) == 0:
                biggestCity = citiesMatch[cityKey]
            else:
                if citiesMatch[cityKey]['population'] > biggestCity['population']:
                    biggestCity = citiesMatch[cityKey]
        df = df.append({'headline' : headline, "city" : biggestCity['name'] , "country" : biggestCity['countrycode']} , ignore_index=True)  
print(df)